# Recommender system - non-personalized keyword-search recommender module

## Introduction ##
A keyword search-based restaurant recommender module is built to filter by keywords of interest. Keyword-search module should support filtering by both restaurant location-based keywords(zip code, longitude, latitude) and restaurant feature-based keywords (cuisine, style, price). The restaurant catalog will be filtered by keywords first, then ranked by the appropriate ranking metrics of user's choice.<br>

## Summary ##

### 1. Laying the groundworks:

#### Calculating geodesic distance:
In order to compute the distance between a restaurant and the location of user's interest, a function needs to be implemented for calculating the geodesic distances between two points on a globe given their coordinates. Two different methods are implemented and compared: <br> 
1. the great circle distance calculated assuming a spherical model of the earth<br>
2. the geodesic distance calculated by Vincenty formula based on a more accurate ellipsoidal model of the earth<br>

The results show that the great-circle distance is reasonably accurate when compared with the geodesic distance calculated from an ellipsoidal model of the earth. Given that calculating great-circle distance is at least 100 times faster on average, therefore the great-circle distance is chosen in this project for calculating the distance give two geocoordinates.

#### Geopy.geocoders for identifying the geocoordinates for the location of interest:
In order to compute the distance between restaurants and user's location of interest using geocoordinates, the 'geocoders' module from the 'geopy' package is selected and tested for translating the location of user's interest in the format of zipcode/city/state into geocoordinates.

#### Adjusted rating:
An adjusted rating score is introduced as an improved metric to replace the original restaurant average star rating.
$$score_i = \frac{\sum_u r_{ui} + k*\mu}{n_i+k}$$
The adjusted score uses the mechanism of the damped mean to regulate the extreme cases of having only a few extreme ratings. k controls the strength of the damping effect: the larger k is, the more actual ratings are required to overcome the global mean. In this case, k is set to 22 (which is the 50% quantile of the review counts for all businesses), but it can be tuned according to various business considerations.<br>

The results indicate that the new metric improves the original rating by taking into consideration both:<br>
1) average rating of the restaurant (indicates the goodness of the restaurant, but does not consider popularity)<br>
2) # of ratings by users (indicates popularity, but does not imply the goodness of the restaurant)<br>

### 2. Implementation & testing of the non-personalized keyword-search recommender module:

**Implementaion of keyword-search recommender module:**<br> 
The restaurant catalog is first filtered by all the keywords indicating user's interests, then ranked by the user-selected ranking criteria. Lastly, the top-n restaurants from the ranked list are returned with the user's choice of n. For the ranking criteria, both the restaurant original average star rating and the newly introduced weighted rating are implemented and available to users via the module interface.<br>

**Testing:**<br>
Module testing shows that the average time of computing and returning the recommendation takes about 1-2 seconds. The returning results only include restaurants matching all keywords.<br>

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

business = pd.read_csv('business_clean.csv')  # contains business data including location data, attributes and categories

# 1. Laying the groundworks

## 1.1 Calculate Geodesic distance between two points on the globe

In [2]:
# calculate geodesic distances between two points on a globe, see https://janakiev.com/blog/gps-points-distance-python/ for more resource
# alternatively, one can use geopy.distance from geopy package to calculate either geodesic or great circle distance, https://github.com/geopy/geopy/blob/master/geopy/distance.py

def great_circle_mile(lat1, lon1, lat2, lon2):
    """
    Compute geodesic distances (great-circle distance) of two points given their coordinates. 
    The function returns the distance in miles. 
    Note: 1. Calculation uses the earth's mean radius of 6371.009 km, 
    2. The central subtended angle is calculated by formula: 
    alpha = cos-1*[sin(lat1)*sin(lat2)+ cos(lat1)*cos(lat2)*cos(lon1-lon2)]
    """
    
    from math import sin, cos, acos, radians
    
    lat1, lon1, lat2, lon2 = radians(lat1), radians(lon1), radians(lat2), radians(lon2) # convert degrees to radians
    earth_radius = 6371.009  # use earth's mean radius in kilometers
    alpha = acos(sin(lat1)*sin(lat2) + cos(lat1)*cos(lat2)*cos(lon1-lon2)) # alpha is in radians
    dis_km = alpha * earth_radius
    dis_mile = dis_km * 0.621371   # convert kilometer to mile
    
    return dis_mile

In [3]:
pos1 = (51.5073219, -0.1276474) # London
pos2 = (52.5170365, 13.3888599) # Berlin
pos3 = (-33.8548157,151.2164539) # Sydney

In [4]:
%%timeit
# great_circle distance
distance_12 = great_circle_mile(pos1[0], pos1[1], pos2[0], pos2[1])
distance_12

1.77 µs ± 27.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [5]:
from geopy.distance import distance

In [6]:
%%timeit
# geodesic distance
distance2_12 = distance(pos1, pos2).miles
distance2_12

206 µs ± 10.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [7]:
distance_12 = great_circle_mile(pos1[0], pos1[1], pos2[0], pos2[1])
distance2_12 = distance(pos1, pos2).miles
error_12 = (distance_12 - distance2_12)/distance2_12

distance_13 = great_circle_mile(pos1[0], pos1[1], pos3[0], pos3[1])
distance2_13 = distance(pos1, pos3).miles
error_13 = (distance_13 - distance2_13)/distance2_13

# print errors, 1-2 is distance between London and Berlin, 1-3 is distance between London and Sydney
print("absolute error:", (distance_12-distance2_12), (distance_13-distance2_13)) 
print("percent error:", error_12, error_13)

absolute error: -1.8326838373754981 2.892945931140275
percent error: -0.0031598293601707256 0.0002740520023695907


### Quick summary: 
Great-circle distance is reasonably accurate when compared with the geodesic distance calculated from an ellipsoidal model of the earth. Given that calculating great-circle distance is at least 100 times faster on average, great-circle distance will be used in this project when calculating geodistance on map.

## 1.2 Adjusted rating

Here, a single metric is introduced as a substitute of the original average restaurant rating ('stars' column of the business dataframe). Ideally, the new metric should take into consideration: <br>
1) average rating of the restaurant (indicates the goodness of the restaurant, but does not consider popularity) <br>
2) # of ratings by users (indicates popularity, but does not imply the goodness of the restaurant) <br>
3) age of the rating (indicates the relevance of the rating, as outdated ratings might fail to indicate the actual quality)<br>

The proposed new score is: 
$$score_i = \frac{\sum_u r_{ui} + k*\mu}{n_i+k}$$

where $r_ui$ is the rating on item i by user u, $n_i$ is the number of rating on item i, $\mu$ is the global mean of ratings over all businesses and all users and k is the strength of the damping term.

As the equation shows, the adjusted score uses the mechanism of the damped mean to regulate the extreme cases of having only a few extreme ratings. k controls the strength of the damping effect: the larger k is, the more actual ratings are required to overcome the global mean.

In this case, k is set to 22 (which is the 50% quantile of the review counts for all businesses), but it can be tuned according to various business considerations.

Note:<br> 
Here, the age of the rating is not adjusted in the current version of the proposed metric. This is because 90% of the reviews/ratings are from year 2011 and later, and are considered quite relevant. Therefore, the need of adjusting for the age of the ratings is not strong.

In [8]:
# compute globe mean ratings of all businesses and all reviews
globe_mean = ((business.stars * business.review_count).sum())/(business.review_count.sum())
print("global mean rating is:", globe_mean)

global mean rating is: 3.7277814701620127


In [9]:
print(business.review_count.quantile([0.1,0.25,0.5,0.75,0.9]))
k = 22 # set strength k to 22, which is the 50% quantile of the review counts for all businesses
business['adjusted_score'] = (business.review_count * business.stars + k * globe_mean)/(business.review_count + k)
print("\nrank by the adjusted score in descending order:")
print(business[['review_count','stars','adjusted_score']].sort_values('adjusted_score', ascending=False).head(5))
print("\nrank by the original score in descending order:")
print(business[['review_count','stars','adjusted_score']].sort_values('stars', ascending=False).head(5))
print("\nrank by the least number of reviews:")
print(business[['review_count','stars','adjusted_score']].sort_values('review_count', ascending=True).head(5))

0.10      4.0
0.25      8.0
0.50     22.0
0.75     66.0
0.90    172.0
Name: review_count, dtype: float64

rank by the adjusted score in descending order:
       review_count  stars  adjusted_score
7464           1746    5.0        4.984169
31910          1380    5.0        4.980037
45401           547    5.0        4.950811
7784            520    5.0        4.948360
28162           472    5.0        4.943342

rank by the original score in descending order:
       review_count  stars  adjusted_score
22115             7    5.0        4.034869
23114             5    5.0        3.963377
42990             5    5.0        3.963377
42989            16    5.0        4.263452
12778             3    5.0        3.880448

rank by the least number of reviews:
       review_count  stars  adjusted_score
0                 3    4.5        3.820448
5707              3    4.0        3.760448
16594             3    4.0        3.760448
5699              3    3.5        3.700448
16605             3    3.5  

## 1.3 Geopy package for querying the geo coordinates

The location of user's interest is typically indicated by zipcode, city, or state, rather than geocoordinates. In order to compute the distance between restaurants and user's location of interest, the zipcode/city/state needs to be translated into geocoordinates first. Luckily, the geopy package has the geocoders module for making geolocation queries by zipcode, city, state or a variety of address formats and returning the geocoordinates. 

In [10]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

# use geolocate query to find the coordinate for the location of interest
geolocator = Nominatim(user_agent="yelp_recommender") # use geopy.geocoders to make geolocation queries
address = "san jose, 95132"

try:
    location = geolocator.geocode(address, timeout=10) 
except GeocoderTimedOut as e:
    print("Error: geocode failed to locate the address of interest {} with message {}".format(address, e.message))            

In [11]:
print(location)
print(location.raw)
print(location.latitude, location.longitude)

San José, Santa Clara County, California, USA
{'place_id': '198749283', 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'relation', 'osm_id': '112143', 'boundingbox': ['37.124503', '37.4692175', '-122.045672', '-121.589153'], 'lat': '37.3361905', 'lon': '-121.8905833', 'display_name': 'San José, Santa Clara County, California, USA', 'class': 'place', 'type': 'city', 'importance': 0.6912737650259659, 'icon': 'https://nominatim.openstreetmap.org/images/mapicons/poi_place_city.p.20.png'}
37.3361905 -121.8905833


note: Using the geopy.geocoders to make geolocation queries sometimes results in 'timeout' error. Setting 'timeout=10' allows making multiple attempts without throwing errors upon just a single attempt, and wrapping with the 'try, except' clause enables an extra layer of protection.

# 2. Building the non-personalized keyword-search recommender module

## 2.1 Implementation

In [12]:
# update data type of the 'postal_code' column of business dataframe to string type
business['postal_code'] = business.postal_code.astype(str)

In [13]:
class Recommender:
    
    def __init__(self, n=5, original_score=False):
        """initiate a Recommender object by passing the desired number of recommendations to make, the default number is 10.
        By default, the adjusted score will be used for ranking; To rank by the original average rating of the restaurant, pass original_score=True
        """
        self.n = n # number of recommendations to make, default is 5
        self.original_score = original_score # boolean indicating whether the original average rating or the adjusted score is used
        # initiate a list of column names to display in the recommendation results
        self.column_to_display = ['state','city','name','address','attributes.RestaurantsPriceRange2','cuisine','style','review_count','stars','adjusted_score']
        
        # initiate the list of recommendations to be all the open restaurants from the entire catalog of 'business' dataframe sorted by the score of interest
        if self.original_score:  # set sorting criteria to the originial star rating
            score = 'stars'
        else:  # set sorting criteria to the adjusted score
            score = 'adjusted_score'
        self.recomm = business[business.is_open == 1].sort_values(score, ascending=False)
        
    def _filter_by_location(self):
        """Filter and update the dataframe of recommendations by the matching location of interest.
        A combination of state, city and zipcode is used as the location information, partially missing information can be handled. 
        Matching restaurant is defined as the restaurant within the acceptable distance (max_distance) of the location of interest.
        note: this hidden method should only be called within the method 'keyword'
        """       
        from geopy.geocoders import Nominatim
        from geopy.exc import GeocoderTimedOut
        geolocator = Nominatim(user_agent="yelp_recommender") # use geopy.geocoders to make geolocation queries
        address = [self.city, self.state, self.zipcode]
        address = ",".join([str(i) for i in address if i != None])
        # use geolocate query to find the coordinate for the location of interest
        try:
            location = geolocator.geocode(address, timeout=10) 
        except GeocoderTimedOut as e:
            print("Error: geocode failed to locate the address of interest {} with message {}".format(address, e.message))            

        # calculate the geodesic distance between each restaurant and the location of interest and add as a new column ''distance_to_interest'
        self.recomm['distance_to_interest'] = self.recomm.apply(lambda row: great_circle_mile(row.latitude, row.longitude, location.latitude, location.longitude), axis=1)
        # add the new column 'distance_to_interest' to the list of columns to display in the recommendation result
        self.column_to_display.insert(0, 'distance_to_interest')
        # filter by the desired distance
        self.recomm = self.recomm[self.recomm.distance_to_interest <= self.max_distance]

    def _filter_by_state(self):
        """ Filter and update the dataframe of recommendations by the matching state.
        note: this hidden method should only be called within the method 'keyword'
        """
        self.recomm = self.recomm[self.recomm.state == self.state]
    
    def _filter_by_cuisine(self):
        """ Filter and update the dataframe of recommendations by the matching cuisine of interest. 
        note: this hidden method should only be called within the method 'keyword'
        """                         
        idx = []
        for i in self.recomm.index: 
            if self.recomm.loc[i,'cuisine'] is not np.nan:
                entries = self.recomm.loc[i,'cuisine'].split(',')
                if self.cuisine in entries:
                    idx.append(i)
        self.recomm = self.recomm.loc[idx]
    
    def _filter_by_style(self):  
        """ Filter and update the dataframe of recommendations by the matching style of interest. 
        note: this hidden method should only be called within the method 'keyword'
        """
        idx = []
        for i in self.recomm.index: 
            if self.recomm.loc[i,'style'] is not np.nan:
                entries = self.recomm.loc[i,'style'].split(',')
                if self.style in entries:
                    idx.append(i)
        self.recomm = self.recomm.loc[idx]
    
    def _filter_by_price(self):
        """Filter and update the dataframe of recommendations by the matching price range of interest. 
        note: this hidden method should only be called within the method 'keyword'
        """
        self.recomm = self.recomm[self.recomm['attributes.RestaurantsPriceRange2'].isin(self.price)]
    
    def display_recommendation(self):
        """ Display the list of top n recommended restaurants
        """
        if len(self.recomm) == 0:
            print("Sorry, there is no matching recommendations.")
        elif self.n < len(self.recomm):  # display only the top n from the recommendation list
            print("Below is a list of the top {} recommended restaurants for you: ".format(self.n))
            print(self.recomm.iloc[:self.n][self.column_to_display])
        else:  # display all if # of recommendations is less than self.n
            print("Below is a list of the top {} recommended restaurants for you: ".format(len(self.recomm)))
            print(self.recomm[self.column_to_display])
    
    # non-personalized keyword filtering recommender module
    def keyword(self, df=business[business.is_open == 1], zipcode=None, city=None, state=None, max_distance=10, cuisine=None, style=None, price=None):
        """Non-personalized recommendation by keyword filtering: 
        Support filtering by the distance and location (zipcode, city, state) of interest, 
        by the desired cuisine, by the desired style, and by the desired price range. 
        The module supports multiple price range inputs separated by comma.
        ---
        Note:
        df: the default restaurant catalog is all the open restaurants in the 'business' dataframe, 
            if a subset is prefered, e.g. previous filtered result, the subset can be passed to df
        state: needs to be the upper case of the state abbreviation, e.g.: 'NV', 'CA'
        max_distance: the max acceptable distance between the restaurant and the location of interest, unit is in miles, default is 10
        ---
        """
        # re-initiate the following variables every time the module is called so that the recommendation starts fresh
        self.recomm = df # start with the desired restaurant catalog
        self.recomm['distance_to_interest'] = np.nan # reset the distance between each restaurant and the location of interest
        self.column_to_display = ['state','city','name','address','attributes.RestaurantsPriceRange2','cuisine','style','review_count','stars','adjusted_score'] # reset the columns to display
        
        # assign variables based on user's keyword inputs
        self.zipcode = zipcode
        self.city = city
        self.state = state
        self.max_distance = max_distance
        self.cuisine = cuisine
        self.style = style
        self.price = price 
             
        # filter by restaurant location
        if (self.zipcode != None) or (self.city != None) or (self.state != None):      
            if (self.zipcode != None) or (self.city != None): # use zipcode and/or city whenever available
                self._filter_by_location()
            else: # filter by state if state is the only location information available
                self._filter_by_state()
            if len(self.recomm) == 0:
                print("no restaurant found for the matching location of interest.")
                return None
        
        # filter by restaurant 'cuisine'
        if self.cuisine != None:
            self._filter_by_cuisine()
            if len(self.recomm) == 0:
                print("no restaurant found for the matching cuisine of {}".format(self.cuisine))
                return None
    
        # filter by restaurant 'style'
        if self.style != None:
            self._filter_by_style() 
            if len(self.recomm) == 0:
                print("no restaurant found for the matching style of {}".format(self.style))
                return None
        
        # filter by restaurant price range
        if self.price != None:
            self.price = [i.strip() for i in price.split(',')] #extract multiple inputs of price range
            self._filter_by_price()
            if len(self.recomm) == 0:
                print("no restaurant found for the matching price of {}".format(self.price))
                return None
        
        # sort the matching list of restaurants by the score of interest
        if self.original_score:  # set sorting criteria to the originial star rating
            score = 'stars'
        else:  # set sorting criteria to the adjusted score
            score = 'adjusted_score'
        self.recomm = self.recomm.sort_values(score, ascending=False)
        
        # display the list of top n recommendations
        self.display_recommendation()
        
        return self.recomm

## 2.2 Testing of the non-personalized keyword filtering recommender module

In [14]:
%%time
# initiate a Recommender object
kw = Recommender(n=3)

# test0: display only (same as no keywords)
print("------\nresult from test0 (display only): ")
kw.display_recommendation()

# test1: no keywords
print("------\nresult from test1 (no keywords): ")
kw.keyword();

# test 2: a combination of city, state and zipcode
print("------\nresult from test2 (a combination of city and state): ")
kw.keyword(city='Phoenix', state='AZ', zipcode='85023');

# test 3: a combination of cuisine and style
print("------\nresult from test3 (a combination of cuisine and style): ")
kw.keyword(cuisine='barbeque', style='restaurants');

# test 4: a combination of state, cuisine and style
print("------\nresult from test4 (a combination of state, cuisine and style): ")
kw.keyword(state='NV', cuisine='desserts', style='restaurants');

# test 5: no matching location
print("------\nresult from test5 (no matching location): ")
kw.keyword(city='milpitas', zipcode='95035');

# test 6: no matching 'cuisine'
print("------\nresult from test6 (no matching cuisine): ")
kw.keyword(cuisine='abc');

# test 7: no matching 'style'
print("------\nresult from test7 (no matching style): ")
kw.keyword(style='abc');

# test 8: a combination of location, cuisine and style
print("------\nresult from test8 (a combination of location, cuisine and style): ")
kw.keyword(city='Phoenix', zipcode='85023',cuisine='barbeque', style='restaurants');

# test 9: a combination of price range, cuisine and style
print("------\nresult from test9 (a combination of price range, cuisine and style): ")
kw.keyword(price='1', cuisine='barbeque', style='restaurants');

# test 10: a combination of two price ranges, location, cuisine and style
print("------\nresult from test10 (a combination of two price ranges, location, cuisine and style): ")
kw.keyword(price='1, 2', zipcode='85023',cuisine='barbeque', style='restaurants');

# test 11: use the original average rating and return top 10 recommendations
print("------\nresult from test11 (top 10 recommendations ranked by original average rating): ")
kw2 = Recommender(n=10, original_score=True)
kw2.keyword(city='Phoenix', zipcode='85023',cuisine='barbeque', style='restaurants');

------
result from test0 (display only): 
Below is a list of the top 3 recommended restaurants for you: 
      state       city             name                       address  \
7464     AZ    Phoenix  Little Miss BBQ          4301 E University Dr   
31910    NV  Las Vegas     Brew Tea Bar  7380 S Rainbow Blvd, Ste 101   
45401    NV  Las Vegas       Gelatology  7910 S Rainbow Blvd, Ste 110   

       attributes.RestaurantsPriceRange2                              cuisine  \
7464                                 2.0                             barbeque   
31910                                1.0                 desserts, bubble tea   
45401                                1.0  ice cream & frozen yogurt, desserts   

                    style  review_count  stars  adjusted_score  
7464          restaurants          1746    5.0        4.984169  
31910  cafes, restaurants          1380    5.0        4.980037  
45401                 NaN           547    5.0        4.950811  
------
result fro

Below is a list of the top 10 recommended restaurants for you: 
       distance_to_interest state     city  \
13651              2.581971    AZ  Phoenix   
44933              1.051736    AZ  Phoenix   
9236               4.579892    AZ  Phoenix   
23589              9.612561    AZ  Phoenix   
25730              7.135294    AZ  Phoenix   
15502              6.433504    AZ  Phoenix   
15693              9.072645    AZ  Phoenix   
44683              6.099196    AZ  Phoenix   
41810              6.020938    AZ  Phoenix   
213                6.571606    AZ   Peoria   

                                         name                        address  \
13651                      Big Cuz's Catering  428 E Thunderbird Rd, Ste 424   
44933                 Pork on a Fork Catering                 1732 W Bell Rd   
9236                                  Bobby Q                8501 N 27th Ave   
23589                         Reathrey Sekong        1312 E Indian School Rd   
25730                   Papa 

As shown, 11 tests (11 queries) are performed with a total CPU time of 8 seconds and elapsed time of 14 seconds. This averages to roughly 1-2 seconds per queries which is very reasonable in practice.